In [ ]:
tils = [
[[0, 0, 0], [0, 1, 1], [1, 1, 1]] ,
[[0, 0, 0], [1, 0, 1], [1, 1, 1]] ,
[[0, 0, 0], [1, 1, 1], [0, 1, 1]] , #0
[[0, 0, 0], [1, 1, 1], [1, 0, 1]] , #1
[[0, 0, 1], [0, 0, 1], [1, 1, 1]] ,
[[0, 0, 1], [0, 1, 0], [1, 1, 1]] ,
[[0, 0, 1], [0, 1, 1], [1, 1, 0]] ,
[[0, 0, 1], [1, 0, 0], [1, 1, 1]] ,
[[0, 0, 1], [1, 0, 1], [0, 1, 1]] ,
[[0, 0, 1], [1, 0, 1], [1, 0, 1]] ,
[[0, 0, 1], [1, 0, 1], [1, 1, 0]] ,
[[0, 0, 1], [1, 1, 0], [0, 1, 1]] ,
[[0, 0, 1], [1, 1, 0], [1, 0, 1]] ,
[[0, 0, 1], [1, 1, 0], [1, 1, 0]] ,
[[0, 0, 1], [1, 1, 1], [0, 0, 1]] ,
[[0, 0, 1], [1, 1, 1], [0, 1, 0]] ,
[[0, 0, 1], [1, 1, 1], [1, 0, 0]] ,
[[0, 1, 0], [1, 0, 1], [0, 1, 1]] ,
[[0, 1, 0], [1, 0, 1], [1, 0, 1]] ,
[[0, 1, 0], [1, 1, 1], [0, 1, 0]] ,
[[0, 1, 1], [1, 0, 0], [1, 0, 1]] ,
[[1, 0, 1], [0, 0, 0], [1, 1, 1]] ,
[[1, 0, 1], [0, 1, 0], [1, 0, 1]] ,
]

dim = (4, 10)
# 3*7*5
import numpy as np
from random import sample

def flatten(t):
    return [item for sublist in t for item in sublist]

def generate_cover(s):
    
    out = []
    a, b = dim[0]-1, dim[1]-1
    a, b = a-2, b-2
    for dx in range(a+1):
         for dy in range(b+1):
            tempA = np.zeros(dim)

            for x in range(3):
                for y in range(3):
                    if s[x][y] == 1:
                        tempA[x+dx][y+dy] = 1
            #print(tempA)
            out.append(list(map(int, tempA.flatten().tolist())))
    return out

def generate_batch(s):
    out = []
    t = np.array(s)
    temparr = []

    for j in range(6):
        t = np.rot90(t)
        if t.tolist() not in temparr:
            temparr.append(t.tolist())
        if np.fliplr(t).tolist() not in temparr:
            temparr.append(t.tolist())

    return flatten(list(map(generate_cover, temparr)))

# t = np.array(generate_cover(tils[0])[0])  
# t.resize((15, 7))
# print(t)

def prep_tiles():
    p = list(map(generate_batch, tils))
    pd = p[:2]
    p = p[2:]
    p[0].extend(pd[0])
    p[1].extend(pd[1])

    for k in range(len(p)):
        d = []
        for i in p[k]:
            if i not in d:
                d.append(i)

        p[k] = d
    
    #print(p[0])
    p = sorted(p, key=len)
    print(len(p))
    print(sum(list(map(len, p))))
    return p

import pysat.solvers
from pysat.card import CardEnc as encode
from random import shuffle, sample

def make_clauses(p):
    from tqdm import tqdm

    intersecting = lambda a, b: any(a[i] == 1 and b[i] == 1 for i in range(len(a)))
    n = sum(list(map(len, p)))
    out = []
    out.append([i for i in range(1, n+1)])
    t = 0
    d = dict()
    for i in range(len(p)):
        for j in range(len(p[i])):
            d[(i, j)] = t
            t+=1

    for i in tqdm(range(len(p))):
        for j in range(len(p[i])):
            # v = -(d[(i, j)]+1)
            temp = []

            for di in range(len(p)):
                for dj in range(len(p[di])):
                    if di == i and dj != j:
                        temp.append([-(d[(i, j)]+1), -1*(d[(di, dj)]+1)])
                    elif (i, j) != (di, dj)  and  intersecting(p[i][j], p[di][dj]):
                        temp.append([-(d[(i, j)]+1), -1*(d[(di, dj)]+1)])

            out.extend(temp)

    
    
    clauses = out

    # for i in range(len(p)):
    #     if i in (17,20):
        # clauses += [[d[(i, j)]+1 for j in range(len(p[i]))]]

    clauses += encode.atleast(lits= [i for i in range(1, n+1)], bound = dim[0]*dim[1]//5).clauses
    #clauses += [[i for i in range(1, 100)]]


    shuffle(clauses)

    solver = pysat.solvers.Cadical()
    for clause in tqdm(clauses):
        solver.add_clause(clause)
    #print('done')
    sat = solver.solve()
    out = []
    k = 0
    
    if sat:
        m = solver.get_model()
        m = [i for i in m if i > 0]

        r = []
        t = 0
        d = dict()
        for i in range(len(p)):
            for j in range(len(p[i])):
                #d[(i, j)] = t
                
                t+=1
                if t in m:
                    r.append(p[i][j])
        out.append( r)
    else:
        pass
        print('unsat')

    return out[0]
cool_ones = []
p = prep_tiles()


c = make_clauses(p)

if c:


#     import numpy as np
#     p = prep_tiles()
#     out = []
#     for a, b in c:
#         out.append(p[a][b])

    solutions = [ c
    ]
    terms = []

    for s in solutions:
        A = np.zeros(dim)
        for k, v in enumerate(s):
            t = np.array(v)
            t.resize(dim)
            #print(t)
            A = A + t*(k+1)
        c = 1
        
        for j in range(5):
            t = np.rot90(t)
            if t.tolist() in terms or  np.fliplr(t).tolist() in  terms:
                c = 0
                break
        if c == 1:
            terms.append(A.tolist())
 
    for i in terms:
        print(i, ',')



21
1300


100%|██████████| 805193/805193 [00:01<00:00, 664557.66it/s]


[[5.0, 5.0, 6.0, 6.0, 6.0, 4.0, 4.0, 4.0, 8.0, 8.0], [5.0, 3.0, 5.0, 7.0, 7.0, 1.0, 4.0, 2.0, 8.0, 8.0], [3.0, 5.0, 6.0, 7.0, 6.0, 1.0, 4.0, 2.0, 2.0, 8.0], [3.0, 3.0, 3.0, 7.0, 7.0, 1.0, 1.0, 1.0, 2.0, 2.0]] ,


In [ ]:
!pip install python-sat[pblib,aiger]

     |████████████████████████████████| 1.8 MB 13.0 MB/s 
     |████████████████████████████████| 3.4 MB 59.9 MB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 
  Created wheel for parsimonious: filename=parsimonious-0.8.1-py3-none-any.whl size=42723 sha256=221dea045935bd7343b09db5f7f6e388f6d623cecdcfd2b18d8894cb8286982a
  Stored in directory: /root/.cache/pip/wheels/88/5d/ba/f27d8af07306b65ee44f9d3f9cadea1db749a421a6db8a99bf
Successfully built parsimonious
